In [56]:
from langchain_groq import ChatGroq

In [57]:
llm = ChatGroq(
    temperature = 0,
    groq_api_key = 'your-api-key',
    model_name = "llama-3.1-70b-versatile"
)

In [58]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://resources.workable.com/ai-engineer-job-description")
page_data = loader.load().pop().page_content
#print(page_data)

In [59]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the careers page of a website.
    Your job is to extract the job postings and return them in JSON fomat containing the
    following keys: `role`, `experience`, `skills`, `responsibilities` and `description`.
    Return the valis JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={"page_data": page_data})
print(res.content)

```json
{
  "role": "AI Engineer",
  "experience": "Degree in Computer Science, Engineering, or related field",
  "skills": [
    "Machine learning",
    "Deep learning",
    "NLP",
    "Computer vision",
    "Python",
    "Java",
    "R",
    "AI frameworks such as TensorFlow or PyTorch"
  ],
  "responsibilities": [
    "Develop machine learning models and AI solutions",
    "Test, deploy, and maintain AI systems",
    "Collaborate with data scientists and other engineers to integrate AI into broader system architectures",
    "Stay current with AI trends and suggest improvements to existing systems and workflows"
  ],
  "description": "We are seeking an AI Engineer to join our dynamic team and contribute to the development and enhancement of our AI-driven platforms. The ideal candidate will possess deep technical expertise in machine learning and artificial intelligence, with a proven track record of developing scalable AI solutions."
}
```


In [60]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)

#converting json_res from a list to a dictionary
if isinstance(json_res, list) and len(json_res) > 0:
    json_res = json_res[0]

json_res

{'role': 'AI Engineer',
 'experience': 'Degree in Computer Science, Engineering, or related field',
 'skills': ['Machine learning',
  'Deep learning',
  'NLP',
  'Computer vision',
  'Python',
  'Java',
  'R',
  'AI frameworks such as TensorFlow or PyTorch'],
 'responsibilities': ['Develop machine learning models and AI solutions',
  'Test, deploy, and maintain AI systems',
  'Collaborate with data scientists and other engineers to integrate AI into broader system architectures',
  'Stay current with AI trends and suggest improvements to existing systems and workflows'],
 'description': 'We are seeking an AI Engineer to join our dynamic team and contribute to the development and enhancement of our AI-driven platforms. The ideal candidate will possess deep technical expertise in machine learning and artificial intelligence, with a proven track record of developing scalable AI solutions.'}

In [61]:
import pandas as pd

df = pd.read_csv("app/resource/my_portfolio.csv")
df


,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [62]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [63]:
job = json_res
job['skills']

['Machine learning',
 'Deep learning',
 'NLP',
 'Computer vision',
 'Python',
 'Java',
 'R',
 'AI frameworks such as TensorFlow or PyTorch']

In [64]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links


[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}]]

In [65]:
job

{'role': 'AI Engineer',
 'experience': 'Degree in Computer Science, Engineering, or related field',
 'skills': ['Machine learning',
  'Deep learning',
  'NLP',
  'Computer vision',
  'Python',
  'Java',
  'R',
  'AI frameworks such as TensorFlow or PyTorch'],
 'responsibilities': ['Develop machine learning models and AI solutions',
  'Test, deploy, and maintain AI systems',
  'Collaborate with data scientists and other engineers to integrate AI into broader system architectures',
  'Stay current with AI trends and suggest improvements to existing systems and workflows'],
 'description': 'We are seeking an AI Engineer to join our dynamic team and contribute to the development and enhancement of our AI-driven platforms. The ideal candidate will possess deep technical expertise in machine learning and artificial intelligence, with a proven track record of developing scalable AI solutions.'}

In [66]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Neeti Kurulkar, the Training and Placement Coordinator at PES Modern College of Engineering. PES Modern College is dedicated to equipping its students with the skills and knowledge needed to excel in today's competitive job market. Our students specialize in various technical fields, including software development, data science, cloud computing, and automation.

        Your job is to write a cold email to potential employers, describing the capabilities and portfolios of PES Modern College's students, and how they can bring value to the organization through internships. Also, add the most relevant portfolios from the following list to showcase the students' achievements: {link_list}

        Remember, you are Neeti Kurulkar, the Training and Placement Coordinator at PES Modern College. Do not provide a preamble.

        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Collaboration Opportunity: Internship Program with PES Modern College of Engineering

Dear Hiring Manager,

I am writing to introduce you to the talented students of PES Modern College of Engineering, who are eager to contribute to innovative projects and gain industry experience through internships. Our students specialize in various technical fields, including software development, data science, cloud computing, and automation, making them an excellent fit for your organization.

We have a pool of skilled students who possess expertise in machine learning, deep learning, NLP, computer vision, and AI frameworks such as TensorFlow or PyTorch. They have developed scalable AI solutions and have a proven track record of delivering high-quality projects.

I would like to bring to your attention the portfolios of our students, which showcase their capabilities in machine learning and AI. Some of their notable projects include:

- Machine Learning with Python: https://example.com/ml